In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

class shape_fns:
    def __init__(self, scale_x = [0, 1], scale_y = [0, 1], p=0):
        self.scale_x = scale_x
        self.scale_y = scale_y
        self.x_l = scale_x[0]
        self.x_r = scale_x[1]
        self.y_l = scale_y[0]
        self.y_r = scale_y[1]
        self.p = p
        
    def expression(self, ksi, neta): 
        return 1-ksi-neta
    
    def __call__(self, x, y):
        x = np.asarray(x)  # convert x to a numpy array if it's not already
        y = np.asarray(y)  # convert y to a numpy array if it's not already
        expression_vectorized = np.vectorize(self.expression, otypes=['d'])
        return np.where((self.x_l <= x) & (x <= self.x_r) & (self.y_l <= y) & (y <= self.y_r), expression_vectorized(x, y), 0)


In [6]:
shape_fn = shape_fns()
x = np.linspace(0, 1, 10)
y = np.linspace(0, 1, 10)
shape_fn(x, y)

array([ 1.        ,  0.77777778,  0.55555556,  0.33333333,  0.11111111,
       -0.11111111, -0.33333333, -0.55555556, -0.77777778, -1.        ])

In [12]:
# compute Jacobian matrix
def jacobian(X, dN):
    # X is the global coordinate for each node.
    # dN is the local coordinate of the phip

    # compute Jacobian matrix
    J = np.dot(np.transpose(X),dN)
    return np.linalg.inv(J), np.linalg.det(J)
X = [[0, 0], [1, 2], [2, 2]]
dN = [[0, 0], [0, 1], [1, 0]]
jacobian(X, dN)

(array([[ 1. , -0.5],
        [-1. ,  1. ]]),
 2.0)

In [87]:
import sympy as sym
import numpy as np
from sympy.matrices import Matrix

# 定义三角形三个顶点的原始坐标 (x0, y0), (x1, y1), (x2, y2)
x0, x1, x2, y0, y1, y2 = sym.symbols(['x_0', 'x_1', 'x_2', 'y_0', 'y_1', 'y_2'])


x0, y0, x1, y1, x2, y2 =  [1, 0.2, 2, 1.3, 0.5, 1.7]
[x0, y0], [x1, y1], [x2, y2] = [[0, 0], [2, 1], [0.5, 2]]
[x0, y0], [x1, y1], [x2, y2] = [[1, 0.9], [2, 1.3], [0.5, 1.7]]



# 定义等参映射的雅可比矩阵
J = Matrix([[x1-x0, y1-y0], 
            [x2-x0, y2-y0]])

# 等腰直角等参单元的坐标 (eta, zeta)
eta, zeta = sym.symbols(['\eta', '\zeta'])
# eta, zeta = [1, 1]
u0 = 1 - eta - zeta
u1 = eta
u2 = zeta

# 对每个顶点计算 [du/deta, du/dzeta]
du = Matrix([[sym.diff(u0, eta),sym.diff(u1, eta),sym.diff(u2, eta)],
[sym.diff(u0, zeta), sym.diff(u1, zeta), sym.diff(u2, zeta)]])

# 对每个顶点，计算 du/dx, du/dy = J^{-1} * [du/deta, du/dzeta]
def deriv_u(): 
    return sym.simplify(J.inv() * du)

# 计算第 e 个单元的 9 个非零矩阵元；省略前面的因子 1/2|J|
Ke = sym.zeros(3,3)
du_ =  deriv_u() * J.det()
for i in range(3):
    for j in range(3):
        Ke[i, j] = du_[:, i].dot(du_[:, j])
Ke        

Matrix([
[ 2.41, -1.07, -1.34],
[-1.07,  0.89,  0.18],
[-1.34,  0.18,  1.16]])

In [82]:
du

Matrix([
[-1, 1, 0],
[-1, 0, 1]])

In [89]:
J.inv()

Matrix([
[0.8, -0.4],
[0.5,  1.0]])

In [70]:
J.det()

3.50000000000000

4.0


In [16]:
class shape_fns:
    def __init__(self, scale_x = [0, 1], scale_y = [0, 1]):
        self.scale_x = scale_x
        self.scale_y = scale_y
        self.x_l = scale_x[0]
        self.x_r = scale_x[1]
        self.y_l = scale_y[0]
        self.y_r = scale_y[1]

    def expression(self, ksi, neta):
        pass  # 这里是你的表达式，具体内容需要你根据需求来填写

    def __call__(self, x, y):
        x = np.asarray(x)  # convert x to a numpy array if it's not already
        y = np.asarray(y)  # convert y to a numpy array if it's not already
        expression_vectorized = np.vectorize(self.expression, otypes=['d'])
        return np.where((self.x_l <= x) & (x <= self.x_r) & (self.y_l <= y) & (y <= self.y_r), expression_vectorized(x, y), 0)

class T3_phi(shape_fns):
    def expression(self, ksi, neta): 
        return ksi + 2 * neta  # 假设你的函数是 ksi + 2 * neta

    def derivative_ksi(self):
        # 对于函数 f(ksi, neta) = ksi + 2 * neta
        # 其在 ksi 方向上的偏导数为 1
        return shape_fns(self.scale_x, self.scale_y)

    def derivative_neta(self):
        # 对于函数 f(ksi, neta) = ksi + 2 * neta
        # 其在 neta 方向上的偏导数为 2
        return shape_fns(self.scale_x, self.scale_y)
    
t3_phi = T3_phi()
derivative_ksi = t3_phi.derivative_ksi()  # 获取在 ksi 方向上的偏导数
derivative_neta = t3_phi.derivative_neta()  # 获取在 neta 方向上的偏导数

derivative_ksi(0, 1)

array(nan)

In [46]:
from scipy import integrate
import numpy as np

# 你的积分函数
def G_integrate_2D(u, N=3, scale = [0,1]):

    a , b = scale # Projection from [-1, 1] to [1, 1]
    x, wx = np.polynomial.legendre.leggauss(N)
    y, wy = np.polynomial.legendre.leggauss(N)
    xp = x*(b-a)/2+(b+a)/2
    wxp = wx*(b-a)/2
    yp = y*(b-a)/2+(b+a)/2
    wyp = wy*(b-a)/2
    integral = 0.0
    for i in range(N):
        for j in range(N):
            integral += wxp[i] * wyp[j] * u(xp[i], yp[j])
            
    return integral

# 定义一个二维函数
def f(x, y):
    return x**3 * y**3
# 使用你的函数计算积分
result = G_integrate_2D(f)
print("The result from your function is: ", result)

# 使用 scipy.integrate 来验证结果
result_scipy, error = integrate.dblquad(f, 0, 1, lambda x: 0, lambda x: 1)
print("The result from scipy.integrate is: ", result_scipy)


The result from your function is:  0.0625
The result from scipy.integrate is:  0.0625
